# Open problems - Single cell perturbations
## Combining gene expression modeling + Limma
#### by Antoine Passemiers

---

In this notebook, we implemented a neural network to estimate gene expression and use Limma to convert predictions to DE values. We prealably saved the normalized read counts, as produced by the `voom` R function from the `limma` package. 

<h1 id="dependencies">Install dependencies and import libraries</h1> <a class="anchor" id="dependencies"></a>

Let's first install R. It is a very time-consuming process, so please be patient.

In [ ]:
import subprocess
subprocess.run(
    'conda install --repodata-fn repodata.json -c conda-forge r-base',
    shell=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

In [ ]:
!pip install rpy2 category_encoders

In [ ]:
import os
import random
import warnings
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.decomposition import KernelPCA
import pandas as pd
import tqdm
import torch

import category_encoders
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
from rpy2.robjects.packages import importr
import rpy2.rinterface as ri
pandas2ri.activate()

In [ ]:
SEED = 0xCAFE

random.seed(SEED)
np.random.seed(SEED)
torch.use_deterministic_algorithms(True)
torch.set_num_threads(1)
torch.manual_seed(SEED)

#### Load data

In [ ]:
#ROOT = os.path.dirname(os.path.abspath(__file__))
ROOT = '..'
DATA_FOLDER = os.path.join(ROOT, 'data')
DATA_FOLDER = '/kaggle/input/open-problems-single-cell-perturbations/'
# EXTERNAL_DATA_FOLDER = os.path.join(ROOT, 'external-data')
EXTERNAL_DATA_FOLDER = '/kaggle/input/op2-expression-data/'

INPUT_CELL_TYPES = ['NK cells', 'T cells CD4+', 'T regulatory cells']
#INPUT_CELL_TYPES = ['NK cells', 'T cells CD4+', 'T cells CD8+', 'T regulatory cells']
OUTPUT_CELL_TYPES = ['Myeloid cells', 'B cells']
CELL_TYPES = INPUT_CELL_TYPES + OUTPUT_CELL_TYPES


df = pd.read_parquet(os.path.join(DATA_FOLDER, 'de_train.parquet'))

cell_types = df['cell_type']
sm_names = df['sm_name']
sm_lincs_ids = df['sm_lincs_id']
all_smiles = df['SMILES']
is_control = df['control']

groups = LabelEncoder().fit_transform(sm_names)

for col_name in ['cell_type', 'sm_name', 'sm_lincs_id', 'SMILES', 'control']:
    df.drop(col_name, axis=1, inplace=True)
data = df.to_numpy(dtype=float)

unique_smiles = list(set(all_smiles))
smiles_dict = {smiles: i for i, smiles in enumerate(unique_smiles)}
unique_cell_types = list(set(cell_types))
unique_sm_names = list(set(sm_names))
cell_type_dict = {cell_type: i for i, cell_type in enumerate(unique_cell_types)}
sm_name_dict = {sm_name: i for i, sm_name in enumerate(unique_sm_names)}
sm_name_dict['Dimethyl Sulfoxide'] = len(sm_name_dict)
gene_names = list(df.columns)
gene_name_dict = {gene_name: i for i, gene_name in enumerate(gene_names)}
sm_name_to_smiles = {sm_name: smiles for sm_name, smiles in zip(sm_names, all_smiles)}
donor_dict = {f'donor_{i}': i for i in range(3)}
plate_name_dict = {f'plate_{i}': i for i in range(6)}
row_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7}

In [ ]:
class NN(torch.nn.Module):
    
    def __init__(self, n_out: int):
        torch.nn.Module.__init__(self)
        
        self.n_in_channels: int = 5
        self.n_in: int = n_out
        self.n_out: int = n_out
        self.encoder = torch.nn.Sequential(
            
            torch.nn.Linear(self.n_in * self.n_in_channels, 128),
            torch.nn.PReLU(128),
            
            torch.nn.Linear(128, 256),
            #torch.nn.BatchNorm1d(256),
            torch.nn.PReLU(256),
            #torch.nn.Dropout(0.2, inplace=False),
            
            torch.nn.Linear(256, 256),
            torch.nn.PReLU(256),
            #torch.nn.Dropout(0.2, inplace=False),
            
            torch.nn.Linear(256, 128),
            torch.nn.PReLU(128),
            #torch.nn.BatchNorm1d(128),
            #torch.nn.Dropout(0.2, inplace=False),
            
            torch.nn.Linear(128, 128),
            torch.nn.PReLU(128),
            
            torch.nn.Linear(128, self.n_out * 5),
        )
        
        self.gene_regressor = torch.nn.Sequential(
            
            torch.nn.Linear(10, 128),
            torch.nn.PReLU(128),
            
            torch.nn.Linear(128, 128),
            torch.nn.PReLU(128),
            
            torch.nn.Linear(128, 1),
        )
        
        self.weight_regressor = torch.nn.Sequential(
            
            torch.nn.Linear(10, 64),
            torch.nn.PReLU(64),
            
            torch.nn.Linear(64, 64),
            torch.nn.PReLU(64),
            
            torch.nn.Linear(64, 1),
            torch.nn.ReLU()
        )

        def init_weights(m):
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.001)
        self.encoder.apply(init_weights)
        self.gene_regressor.apply(init_weights)
        self.weight_regressor.apply(init_weights)
        
    def forward(self, X: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        encoded = self.encoder(X.reshape(len(X), -1)).reshape(len(X), -1, 5)
        encoded = torch.cat((encoded, X), dim=2)
        Y = self.gene_regressor(encoded.reshape(-1, encoded.size()[2])).reshape(len(X), -1)
        W = self.weight_regressor(encoded.reshape(-1, encoded.size()[2])).reshape(len(X), -1)
        return Y, W
    
    def predict(self, X: np.ndarray, n_batches: int = 100) -> Tuple[np.ndarray, np.ndarray]:
        X = torch.FloatTensor(X)
        Y_pred_s, W_pred_s = [], []
        for idx in np.array_split(np.arange(len(X)), n_batches):
            Y_pred, W_pred = self.forward(X[idx, :])
            Y_pred_s.append(Y_pred.cpu().data.numpy())
            W_pred_s.append(W_pred.cpu().data.numpy())
        Y_pred = np.concatenate(Y_pred_s, axis=0)
        W_pred = np.concatenate(W_pred_s, axis=0)
        return Y_pred, W_pred

#### Target encoding

In [ ]:
df_s, y_s, weights_s = [], [], []
for cell_type in CELL_TYPES:
    cell_type = cell_type.replace('+', '')
    meta_df = pd.read_csv(os.path.join(EXTERNAL_DATA_FOLDER, f'meta-{cell_type}.tsv'), delimiter='\t', header='infer')
    r_data = np.load(os.path.join(EXTERNAL_DATA_FOLDER, f'{cell_type}.npz'))
    y = r_data['expression']  # Gene expression values, of shape (n_samples, n_genes)
    weights = r_data['weights']  # Inverse variance weights, of shape (n_samples, n_genes)
    df_s.append(meta_df)
    y_s.append(y)
    weights_s.append(weights)
meta_df = pd.concat(df_s, axis=0)
y = np.concatenate(y_s, axis=0)
weights = np.concatenate(weights_s, axis=0)

meta_idx = {key: i for i, key in enumerate(zip(*[meta_df[col_name] for col_name in ['sm_name', 'cell_type', 'donor_id', 'plate_name', 'row']]))}

In [ ]:
class MultiOutputTargetEncoder:
    """Multi-output target encoder.
    
    Each input (categorical) feature will be encoded based on each (continuous) output variable.
    
    Attributes:
        encoders: List of encoders, of shape `n_genes`.
    """
    
    def __init__(self):
        self.encoders: List[category_encoders.leave_one_out.LeaveOneOutEncoder] = []
        
    @staticmethod
    def new_encoder() -> category_encoders.leave_one_out.LeaveOneOutEncoder:
        """Instantiates a new gene-specific target encoder.
        
        Returns:
            New instance of a target encoder.
        """
        return category_encoders.leave_one_out.LeaveOneOutEncoder(return_df=False)
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """Fit the encoders for each input feature and output variable.
        
        Args:
            X: Array of shape `(n, n_features)` containing categories as strings or integers.
                Typical, `n_features` is equal to 2 (cell type + compound).
            y: Array of shape `(n, n_genes)` containing the DE values for all the genes.
        """
        self.encoders = []
        for j in tqdm.tqdm(range(y.shape[1]), desc='fit LOO encoders'):
            self.encoders.append(MultiOutputTargetEncoder.new_encoder())
            self.encoders[-1].fit(X, y[:, j])
    
    def transform(self, X: np.ndarray) -> np.ndarray:
        """Encodes the categories. Assumes the encoders have already been fitted.
        
        Args:
            X: Array of shape `(n, n_features)` containing categories as strings or integers.
        
        Returns:
            Array of shape `(n, n_genes, n_features)` containing the encoding of each input
                feature with respect to each output variable.
        """
        Z = []
        for encoder in tqdm.tqdm(self.encoders, desc='transform LOO encoders'):
            y_hat = encoder.transform(X)
            Z.append(y_hat)
        Z = np.asarray(Z)
        return np.transpose(Z, (1, 0, 2))

In [ ]:
scaler = RobustScaler()
y_scaled = scaler.fit_transform(y)

In [ ]:
encoder = MultiOutputTargetEncoder()
encoder.fit(meta_df[['sm_name', 'donor_id', 'plate_name', 'row', 'cell_type']], y_scaled)

In [ ]:
X = encoder.transform(meta_df[['sm_name', 'donor_id', 'plate_name', 'row', 'cell_type']])
X = torch.FloatTensor(X)

In [ ]:
def train(idx_train):
    idx_train = np.copy(idx_train)
    model = NN(len(gene_names))

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, eps=1e-7)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.9, patience=10, verbose=False, threshold=0.0001,
            threshold_mode='rel', cooldown=2, min_lr=1e-5, eps=1e-08)
    pbar = tqdm.tqdm(range(10))
    for epoch in pbar:
        total_loss, baseline_total_loss = 0, 0
        np.random.shuffle(idx_train)
        for it_idx in np.array_split(idx_train, 500):
            optimizer.zero_grad()
            
            Y = torch.FloatTensor(y_scaled[it_idx, :])
            W = torch.FloatTensor(weights[it_idx, :])
            W = torch.clamp(W, 0, 100)
            #Y = Y + 0.5 * torch.randn(*Y.size())
            Y_pred, W_pred = model.forward(X[it_idx])
            mrrmse = torch.mean(torch.sqrt(torch.mean(torch.square(Y - Y_pred), dim=1)))
            baseline_mrrmse = torch.mean(torch.sqrt(torch.mean(torch.square(Y - torch.mean(Y, dim=0).unsqueeze(0)), dim=1)))
            loss = torch.mean(torch.square(Y - Y_pred))
            loss = loss + 0.5 * torch.mean(torch.square((W - W_pred) / 100))
            loss.backward()
            optimizer.step()
            total_loss += mrrmse.item()
            baseline_total_loss += baseline_mrrmse.item()
        rel_error = total_loss / baseline_total_loss
        scheduler.step(rel_error)    
        pbar.set_description(f'{rel_error:.3f}')
    return model

In [ ]:
models = []
rel_errors = []
kf = KFold(n_splits=10, random_state=SEED, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(y)):
    
    # Train model
    model = train(train_index)
    model.eval()
    models.append(model)

    # Predict on held-out samples
    Y = torch.FloatTensor(y_scaled[test_index, :])
    Y_pred, W_pred = model.forward(X[test_index])
    
    plt.scatter(Y[0, :].cpu().data.numpy(), Y_pred[0, :].cpu().data.numpy(), alpha=0.1)
    plt.show()
    
    #mrrmse = torch.mean(torch.abs(Y - Y_pred)).item()
    #baseline_mrrmse = torch.mean(torch.abs(Y - torch.mean(Y, dim=0).unsqueeze(0))).item()
    mrrmse = torch.mean(torch.sqrt(torch.mean(torch.square(Y - Y_pred), dim=1))).item()
    baseline_mrrmse = torch.mean(torch.sqrt(torch.mean(torch.square(Y - torch.mean(Y, dim=0).unsqueeze(0)), dim=1))).item()
    rel_errors.append(mrrmse / baseline_mrrmse)
    print(rel_errors[-1])
    
print(f'Average relative error: {np.mean(rel_errors)}')

# Average relative error: 0.6952671768202316
# Average relative error: 0.6932183754092522

In [ ]:
model = train(np.arange(len(y)))

---

## Make submission

---

In [ ]:
class Limma:
    
    r_utils = importr('utils')
    r_utils.chooseCRANmirror(ind=1)
    r_utils.install_packages('BiocManager')
    r_biocmanager = importr('BiocManager')
    r_biocmanager.install('limma')
    r_limma = importr('limma')
    r_code = """
    library(limma)
    create_elist <- function(expression, weights) {
        E <- new("EList")
        E$E <- expression
        E$weights <- weights
        return(E)
    }
    make_design <- function(df) {
        for (column in colnames(df)){
            assign(column, df[[column]])
        }
        mm <- model.matrix(eval(parse(text="~0+sm_name+donor_id+plate_name+row")))
        return(mm)
    }
    """
    r_package = SignatureTranslatedAnonymousPackage(r_code, 'custom')
    
    @staticmethod
    def run(df: pd.DataFrame, y: np.ndarray, weights: np.ndarray) -> np.ndarray:
        """Run Limma to obtain the DE values.
        
        Args:
            df: Pandas dataframe containing 4 columns, namely "compound", "donor_id", "plate_name" and "row".
            y: Gene expression values, a matrix of shape (n_samples, n_genes). The number of rows in `df` is 
                equal to the number of rows in `y`.
            weights: Inverse variance weights, a matrix of shape (n_samples, n_genes). The dimensions of `weights`
                should match the dimensions of `y`.
        
        Returns:
            A matrix of shape (n_contrasts, n_genes), where `n_contrasts` is the number of compounds after
                excluding "Dimethyl Sulfoxide".
        """
        
        # Create design matrix and targets
        X = Limma.r_package.make_design(df)
        yw = Limma.r_package.create_elist(
            rpy2.robjects.r.matrix(y.T, nrow=y.T.shape[0], ncol=y.T.shape[1]),
            rpy2.robjects.r.matrix(weights.T, nrow=weights.T.shape[0], ncol=weights.T.shape[1])
        )
        print(f'Design matrix: {X.shape}')

        # Build contrast matrix, also keep track in `contrast_dict` of where each compound
        # is located in design matrix
        sm_name_to_col_idx = {}
        for p in range(len(X)):
            sm_name = df.loc[p, 'sm_name']
            j = np.where(X[p, :])[0][0]
            if sm_name not in sm_name_to_col_idx:
                sm_name_to_col_idx[sm_name] = j
            else:
                assert sm_name_to_col_idx[sm_name] == j
        C = np.zeros((X.shape[1], len(sm_name_to_col_idx) - 1))
        p = 0
        contrast_dict = {}
        unique_sm_names = set(list(df['sm_name']))
        for sm_name in unique_sm_names:
            if sm_name == 'Dimethyl Sulfoxide':
                continue
            if sm_name not in sm_name_to_col_idx:
                continue
            i = sm_name_to_col_idx[sm_name]
            j = sm_name_to_col_idx['Dimethyl Sulfoxide']
            C[i, p] = 1
            C[j, p] = -1
            contrast_dict[sm_name] = p
            p += 1
        print(f'Contrast matrix: {C.shape}')
        
        # Fit linear models
        print(f'Running linear models.')
        fit = Limma.r_limma.lmFit(
            yw,
            rpy2.robjects.r.matrix(X, nrow=X.shape[0], ncol=X.shape[1])
        )
        
        # Replace covariates by their contrasts
        print(f'Computing contrasts.')
        fit = Limma.r_limma.contrasts_fit(
            fit,
            rpy2.robjects.r.matrix(C, nrow=C.shape[0], ncol=C.shape[1])
        )
        
        # Use an empirical Bayes approach to squeeze the residual variances
        print(f'Running empirical Bayes.')
        fit = Limma.r_limma.eBayes(fit)
        
        # Compute DE values
        de = -np.sign(fit.rx2('t')) * np.log10(fit.rx2('p.value'))
        print(f'Done.')
        
        return de, contrast_dict

In [ ]:
def prepare_data(cell_type, model):
    Y, W = [], []
    compounds, plates, donors, rows, cell_types = [], [], [], [], []
    for plate in [f'plate_{i}' for i in range(6)]:
        for donor in [f'donor_{i}' for i in range(3)]:
            for row in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
                for compound in sm_name_dict.keys():
                    """
                    if (compound, cell_type, donor, plate, row) in meta_idx:
                        i = meta_idx[(compound, cell_type, donor, plate, row)]
                        Y.append(y[i, :])
                        W.append(weights[i, :])
                        ok = True
                    """
                    if random.random() < 0.1:
                        Y.append(np.full(len(gene_names), np.nan))
                        W.append(np.full(len(gene_names), np.nan))
                        ok = True
                    else:
                        ok = False
                    if ok:
                        compounds.append(compound)
                        plates.append(plate)
                        donors.append(donor)
                        rows.append(row)
                        cell_types.append(cell_type)
    Y = np.asarray(Y)
    W = np.asarray(W)
                        
    df = pd.DataFrame({
        'sm_name': np.asarray(compounds),
        'donor_id': np.asarray(donors),
        'plate_name': np.asarray(plates),
        'row': np.asarray(rows),
        'cell_type': np.asarray(cell_types)
    })
    print(df)
    X = torch.FloatTensor(encoder.transform(df))
    Y_pred, W_pred = model.predict(X)
    assert Y.shape == Y_pred.shape
    assert W.shape == W_pred.shape
    
    Y_pred = scaler.inverse_transform(Y_pred)
    
    print(len(df), Y.shape, W.shape)
    
    return df, Y_pred, W_pred
    

all_de_pred = {}
for cell_type in OUTPUT_CELL_TYPES:
    df, Y, W = prepare_data(cell_type, model)
    de, contrast_dict = Limma.run(df, Y, W)
    de[np.isnan(de)] = 0
    de[np.isinf(de)] = 0
    all_de_pred[cell_type] = (de, contrast_dict)

In [ ]:
# Load ID mapping
id_map = []
with open(os.path.join(DATA_FOLDER, 'id_map.csv'), 'r') as f:
    lines = f.readlines()[1:]
    for line in lines:
        id_map.append(line.rstrip().split(','))
        assert len(id_map[-1]) == 3

# Make submission
with open('submission.csv', 'w') as f:
    f.write(f'id,{",".join(gene_names)}\n')
    for id_, cell_type, sm_name in tqdm.tqdm(id_map, desc='Submission'):

        # Predict on test sample
        de, contrast_dict = all_de_pred[cell_type]
        i = contrast_dict[sm_name]
        y_hat = np.nan_to_num(de[:, i])
        y_hat = np.clip(y_hat, -100, 100)

        # Write predictions in output file
        values = [f'{x:.5f}' for x in y_hat]
        f.write(f'{id_},{",".join(values)}\n')
